# Set-Up

Installations and imports:

In [ ]:
fenics_dir = '/root/shared/fenics_models'
# !pip install seaborn matplotlib
import os
from math import ceil
import time
import json
import numpy as np
import sys
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
sys.path.insert(0, fenics_dir)
import fenics_helpers

Plotting settings:

In [ ]:
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})
mpl.rcParams['figure.dpi'] = 100
mpl_save_dpi = 300

Save settings:

In [ ]:
fig_save_dir = os.path.join(fenics_dir, 'figs')
if not os.path.isdir(fig_save_dir):
    os.mkdir(fig_save_dir)
data_save_dir = os.path.join(fenics_dir, 'data')
if not os.path.isdir(data_save_dir):
    os.mkdir(data_save_dir)

# Linear Beam Simulations

We shall assume that the mesh size and $\kappa$ parameter values found by simulating the neo-Hookean beam in Notebook \[1\] are sufficiently large to ensure convergence in the case of the linear beam; thus, our fixed parameters are:

In [ ]:
fixed_params = {# Parameters found by convergence studies in Notebook [1]:
                'NW': 6,
                'kappa': 3000, # in kPa
                # Physical parameters:
                'density': 0.00102, # in g/mm^3
                'g': 9.81, # in m/s^3
               # Beam mesh/geometry parameters:
                'width': 40, # in mm
                'length': 90, # in mm
                'elem_order': 2,
                # Linear solver parameters:
                'rtol': 1e-9, 
                'atol': 1e-9, 
                'max_iter': 50}
fixed_params['elem_size'] =  fixed_params['width']/fixed_params['NW']
fixed_params['NL'] =  ceil(fixed_params['length']/fixed_params['elem_size'])

We can now create our mesh:

In [ ]:
mesh = fenics_helpers.create_cuboidal_mesh(*[fixed_params[key] for key in ('length', 'width', 'NL', 'NW')])

Define grid points for training data evaluations:

In [ ]:
num_train_pts = 10 # number of training points per dimension of grid
min_C_1, max_C_1 = 1.9, 6.9 # in kPa
C_1_list = [C_1 for C_1 in np.linspace(min_C_1, max_C_1, num_train_pts)]
min_angle, max_angle = 0, 180 # in degrees
beam_angle_list = [y for y in np.linspace(min_angle, max_angle, num_train_pts)]
training_param_combos = fenics_helpers.create_param_combos(C_1=C_1_list, beam_angle=beam_angle_list)

Define grid points for test data evaluations:

In [ ]:
num_test_pts = num_train_pts-1
delta_C_1 = (max_C_1-min_C_1)/num_test_pts
delta_angle = (max_angle-min_angle)/num_test_pts
C_1_list = [C_1 for C_1 in np.linspace(min_C_1+0.5*delta_C_1, max_C_1-0.5*delta_C_1, num_test_pts)]
beam_angle_list = [angle for angle in np.linspace(min_angle+0.5*delta_angle, max_angle-0.5*delta_angle, num_test_pts)]
test_param_combos = fenics_helpers.create_param_combos(C_1=C_1_list, beam_angle=beam_angle_list)

Compute training data:

In [ ]:
results = {key: [] for key in ('C_1', 'beam_angle', 'end_disp', 't_solve', 'volume')}
for i, params in enumerate(training_param_combos):

    print(f"Simulating Mesh {i+1}/{len(elem_size_list)} (C_1 = {params['C_1']}, Beam angle = {params['beam_angle']})")
    results['C_1'].append(params['C_1'])
    results['beam_angle'].append(params['beam_angle'])
    
    # Note how long each simulation takes:
    t_start = time.time()
    u = fenics_helpers.simulate_linear_beam(mesh, C_1=params['C_1'], beam_angle=params['beam_angle'], **fixed_params)
    t_solve = time.time() - t_start
    results['t_solve'].append(t_solve)
    print(f'Simulation took {t_solve/60:.2f} mins.\n')
    
    results['end_disp'].append(fenics_helpers.compute_end_displacement(u, mesh, fixed_params['width'], fixed_params['length']))
    results['volume'].append(fenics_helpers.compute_pre_and_postdeformation_volume(u, mesh))

results['fixed_params'] = fixed_params

# Save results to json:
with open('linear_beam_training_data.json', 'w') as f:
    json.dump(results, f, indent=4)

Compute test data:

In [ ]:
results = {key: [] for key in ('C_1', 'beam_angle', 'end_disp', 't_solve', 'volume')}
for i, params in enumerate(test_param_combos):

    print(f"Simulating Mesh {i+1}/{len(elem_size_list)} (C_1 = {params['C_1']}, Beam angle = {params['beam_angle']})")
    results['C_1'].append(params['C_1'])
    results['beam_angle'].append(params['beam_angle'])
    
    # Note how long each simulation takes:
    t_start = time.time()
    u = fenics_helpers.simulate_linear_beam(mesh, C_1=params['C_1'], beam_angle=params['beam_angle'], **fixed_params)
    t_solve = time.time() - t_start
    results['t_solve'].append(t_solve)
    print(f'Simulation took {t_solve/60:.2f} mins.\n')
    
    results['end_disp'].append(fenics_helpers.compute_end_displacement(u, mesh, fixed_params['width'], fixed_params['length']))
    results['volume'].append(fenics_helpers.compute_pre_and_postdeformation_volume(u, mesh))

results['fixed_params'] = fixed_params

# Save results to json:
with open('linear_beam_test_data.json', 'w') as f:
    json.dump(results, f, indent=4)